In [8]:
using DynamicPolynomials, SparseArrays

println("***Problem setting***")

n=5

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    lv=length(v)
    c=2*rand(Float64,lv).-1
    return sum(c[j]*v[j] for j in 1:lv)
end
# random quadratic objective function f
v=reverse(monomials(x,2))
f=generate_random_poly(v)


# unit sphere constraint
g=[1.0-sum(x.^2)] #type of coefficients of each polynomial must be float

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/4)

h=Vector{Polynomial{true,Float64}}(undef,l)
randx=2*rand(n).-1# create a feasible solution
randx=rand(1)[1]*randx./sqrt(sum(randx.^2))


for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)


***Problem setting***
Number of variable: n=5
Number of inequality constraints: m=1
Number of equality constraints: l=2


In [9]:
using ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=false);

In [10]:
k=2

println("Relaxed order: k=",k)
println("====================")
t=1
println("Sparse order: t=",t)

Relaxed order: k=2
Sparse order: t=1


In [11]:
using ctpPOP

opt_val=ctpPOP.POP_TS_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
                         tol=1e-3,use_eqcons_to_get_constant_trace=false)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=4
  Size of the largest block: s^max=16
  Number of equality trace constraints: zeta=114
Modeling time:
  0.007562 seconds (31.42 k allocations: 2.486 MiB)
---------------
| Parameters: |
---------------
n:       114
maxtime: 300000.000000
na:      2
mcu:     5
mc:      7
rpar: 
ipar: 
 Entry to LMBM:
 NIT=    1  NFE=    1  F= 0.37472779D+00  WK= 0.3551D+00  QK= 0.1776D+00
 NIT=    2  NFE=    2  F= 0.19827788D+00  WK= 0.2049D+00  QK= 0.2121D+00
 NIT=    3  NFE=    3  F= 0.19827788D+00  WK= 0.1354D+00  QK= 0.1053D+00
 NIT=    4  NFE=    4  F= 0.13798671D+00  WK= 0.7426D-01  QK= 0.5583D-01
 NIT=    5  NFE=    5  F= 0.13798671D+00  WK= 0.1117D+00  QK= 0.5583D-01
 NIT=    6  NFE=    6  F= 0.13798671D+00  WK= 0.6132D-01  QK= 0.3072D-01
 NIT=    7  NFE=    7  F= 0.12266149D+00  WK= 0.6486D-01  QK= 0.1804D+00
 NIT=    8  NFE=    8  F= 0.11953973D+00  WK= 0.4452D-01  QK= 0.2098D+00
 NIT=    9  NFE= 

-0.5758483049027132

In [12]:
using TSSOS

@time begin
    ~,~,data=tssos_first([[f];g;h],x,k,numeq=length(h),TS="block",quotient=false);
    for j in 1:t-1
        ~,~,data=tssos_higher!(data,TS="block")
    end
end

------------------------------------------------------
The sizes of blocks:
[16, 5]
[1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 86              
  Cones                  : 0               
  Scalar variables       : 34              
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  

In [13]:
using ctpPOP

opt_val=ctpPOP.POP_TS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
                         tol=1e-3,use_eqcons_to_get_constant_trace=false)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=4
  Size of the largest block: s^max=16
  Number of equality trace constraints: zeta=114
Modeling time:
  0.006835 seconds (31.42 k allocations: 2.486 MiB)
iter=1   val=-2.313144240624402   gap=2.6807719652215836e-16   feas=3.493811337711493
iter=2   val=-0.7710480802081341   gap=0.530990689311651   feas=0.904745053033807
iter=4   val=-0.6184081269206378   gap=0.5003690392366831   feas=0.6467589245406962
iter=8   val=-0.5191613278549563   gap=0.21666540520062058   feas=0.4218994132098906
iter=16   val=-0.6190157195589001   gap=0.2025246745727649   feas=0.3052573096878286
iter=32   val=-0.5918342625705112   gap=0.160435882142024   feas=0.16407381767618742
iter=64   val=-0.5488051197411029   gap=0.11942523233005596   feas=0.08670932109906114
iter=128   val=-0.5461700347295371   gap=0.05976535354113821   feas=0.04499675134722371
iter=256   val=-0.5132129371169735   gap=0.05435580896387246   fea

-0.49943517094886186

In [14]:
#using ctpPOP

#optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)